In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()["Outcome"]

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
X = df.drop("Outcome", axis = 1)
y = df["Outcome"]

In [6]:
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [7]:
y.tail()

763    0
764    0
765    0
766    1
767    0
Name: Outcome, dtype: int64

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [9]:
X= scaler.fit_transform(X)
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 1)

In [11]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Dense(32, activation = "relu", input_dim = 8))
model.add(Dense(1,activation = "sigmoid"))

model.compile(optimizer = "adam", metrics = ['accuracy'], loss = "binary_crossentropy")

history = model.fit(X_train,y_train, epochs = 100, batch_size = 32, validation_data = (X_test,y_test))

Performing Hyperparameter tuning


In [ ]:
!pip install keras-tuner

In [44]:
import keras_tuner as kt

Chossing best optimizers


In [102]:
def build_model(hp):
    model = keras.Sequential()
    model.add(Dense(
            # Define the hyperparameter.
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            activation="relu",
        )
    )
    model.add(Dense(1, activation="sigmoid"))

    optimizer = hp.Choice('optimizer',values = ["rmsprop",'adadelta', 'adam'])
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"],
    )
    return model

build_model(kt.HyperParameters())

In [107]:
# tuner = kt.RandomSearch(
#     hypermodel=build_model,
#     objective="val_accuracy",
#     max_trials=3,
#     directory="my_dir",
#     project_name="trail",
# )
tuner = kt.Hyperband(
    hypermodel = build_model,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2,
    directory="Keras_tuner_dir",
    project_name="Keras_tuner_Demo")

In [108]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 61 Complete [00h 00m 02s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 01m 50s


In [109]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adadelta', 'adam'], 'ordered': False}


In [110]:
tuner.get_best_hyperparameters()[0].values

{'units': 480,
 'optimizer': 'rmsprop',
 'tuner/epochs': 30,
 'tuner/initial_epoch': 10,
 'tuner/bracket': 3,
 'tuner/round': 3,
 'tuner/trial_id': '0047'}

In [ ]:
model = tuner.get_best_models(1)[0]
model.build(X_train.shape)

In [118]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (614, 480)                4320      
                                                                 
 dense_1 (Dense)             (614, 1)                  481       
                                                                 
Total params: 4,801
Trainable params: 4,801
Non-trainable params: 0
_________________________________________________________________


In [119]:
model.fit(X_train,y_train, epochs = 100, batch_size = 32, validation_data = (X_test,y_test), initial_epoch = 6)

Epoch 7/100
20/20 [==============================] - 1s 16ms/step - loss: 0.4208 - accuracy: 0.8046 - val_loss: 0.4623 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4199 - accuracy: 0.8029 - val_loss: 0.4659 - val_accuracy: 0.7857
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4175 - accuracy: 0.8029 - val_loss: 0.4661 - val_accuracy: 0.8117
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4168 - accuracy: 0.8029 - val_loss: 0.4644 - val_accuracy: 0.8117
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4162 - accuracy: 0.8013 - val_loss: 0.4716 - val_accuracy: 0.8052
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4144 - accuracy: 0.8094 - val_loss: 0.4645 - val_accuracy: 0.7922
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4129 - accuracy: 0.8013 - val_loss: 0.4623 - val_accuracy: 0.8052
Epoch 14

In [120]:
optimizer = model.optimizer
print(optimizer)